In [ ]:
!pip install gigachain

In [ ]:
!pip install chromadb --quiet
!pip install -U unstructured
!pip install unstructured[pdf]

In [ ]:
given_auth_data = "Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA=="

In [ ]:
from langchain.chat_models import GigaChat
from langchain_community.document_loaders import OnlinePDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
url = "https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf" # paper url

In [ ]:
question_en = "What kind of problems there are in sequence transduction deep learning models?"

question_ru = "Какие нерешённые проблемы остались у моделей глубого обучения для обработки последовательностей?"

In [ ]:
class ProblemSeekerAgent:
  def __init__(self, auth_data):
    self.llm = GigaChat(credentials=auth_data, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')
    self.embeddings = GigaChatEmbeddings(credentials=auth_data, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')
    self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

  def read_article(self, url):
    loader = OnlinePDFLoader(url)
    documents = loader.load()
    documents = self.text_splitter.split_documents(documents)
    db = Chroma.from_documents(documents, self.embeddings, client_settings=Settings(anonymized_telemetry=False))
    self.db = db

  def generate_answer(self, question):
    self.qa_chain = RetrievalQA.from_chain_type(self.llm, retriever=self.db.as_retriever())
    return self.qa_chain({"query": question})['result']

In [ ]:
problem_seeker = ProblemSeekerAgent(given_auth_data)

problem_seeker.read_article(url)

answer = problem_seeker.generate_answer(question_ru)

print(answer)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


У моделей глубокого обучения для обработки последовательностей осталось несколько нерешенных проблем. Одной из них является ограничение на последовательный характер вычислений, которое делает невозможным параллельное выполнение операций внутри одного примера. Это становится особенно критичным при работе с длинными последовательностями, поскольку ограничения по памяти ограничивают возможность объединения примеров в одну группу для вычислений. Другой проблемой является ограниченная эффективность моделей при обработке зависимостей между элементами, расположенными на большом расстоянии друг от друга. В некоторых моделях, таких как Extended Neural GPU, ByteNet и ConvS2S, количество операций, необходимых для связи сигналов от двух произвольных элементов, увеличивается с увеличением расстояния между ними. Это затрудняет обучение зависимостей между удаленными элементами.
